# A short introduction to containerized software

After spending time using nf-core pipelines to answer bioinformatic questions, we will focus on the processes that lie behind these pipelines now.

Today, we will focus on containerization, namely via Docker. 



1. Check if Docker is installed.

In [ ]:
!docker info

### What is a container?

### Why do we use containers?

### What is a docker image?

### Let's run our first docker image:

### Login to docker

In [ ]:
# This you need to do on the command line directly

### Run your first docker container

In [ ]:
!docker run hello-world

### Find the container ID

### Delete the container again, give prove its deleted

### FASTQC is a very useful tool as you've learned last week. Let's try and run it from command line

Link to the software: https://www.bioinformatics.babraham.ac.uk/projects/fastqc/

Please describe the steps you took to download and run the software for the example fastq file from last week below:

1. 
2.
...

### Very well, now let's try to make use of its docker container

1. create a container holding fastqc using seqera containers (https://seqera.io/containers/)
2. use the container to generate a fastqc html of the example fastq file

In [ ]:
# pull the container


In [ ]:
# run the container and save the results to a new "fastqc_results" directory

### Now that you know how to use a docker container, which approach was easier and which approach will be easier in the future?

### What would you say, which approach is more reproducible?

### Compare the file to last weeks fastqc results, are they identical?
### Is the fastqc version identical?

## Dockerfiles

We now used Docker containers and images directly to boost our research. 

Let's create our own toy Dockerfile including the "cowsay" tool (https://en.wikipedia.org/wiki/Cowsay)

Hints:
1. Docker is Linux, so you need to know the apt-get command to install "cowsay"

In [ ]:
# open the file "my_dockerfile" in a text editor

### Explain the RUN and ENV lines you added to the file

In [4]:
# build the docker image


In [ ]:
# make sure that the image has been built


In [ ]:
# run the docker file 


## Let's do some bioinformatics with the docker file and create a new docker file that holds the salmon tool used in rnaseq

To do so, use "curl" in your new dockerfile to get salmon from https://github.com/COMBINE-lab/salmon/releases/download/v1.5.2/salmon-1.5.2_linux_x86_64.tar.gz

In [ ]:
# use the file "salmon_docker" in this directory to build a new docker image

In [ ]:
# build the image


In [1]:
# run the docker image to give out the version of salmon


## Do you think bioinformaticians have to create a docker image every time they want to run a tool?

Find the salmon docker image online and run it on your computer.

What is https://biocontainers.pro/ ?

## Are there other ways to create Docker (or Apptainer) images?

What is https://seqera.io/containers/ ?